<a href="https://colab.research.google.com/github/DorotaJanosz/neural-network-course/blob/master/02_kears/03_overfitting_underfitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import bibliotek

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.datasets.imdb import get_word_index
from tensorflow.keras.utils import get_file
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

sns.set()
tf.__version__


'2.6.0'

### Załadowanie danych

In [19]:
NUM_WORDS = 10000   # 10000 najczęściej pojawiających się słów
INDEX_FROM = 3

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=NUM_WORDS, index_from=INDEX_FROM)

In [20]:
print(f'Train data shape: {train_data.shape}')
print(f'Test data shape: {test_data.shape}')

Train data shape: (25000,)
Test data shape: (25000,)


In [21]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [22]:
word_to_idx = get_word_index()
word_to_idx = {k:(v + INDEX_FROM) for k, v in word_to_idx.items()}
word_to_idx['<PAD>'] = 0
word_to_idx['<START>'] = 1
word_to_idx['<UNK>'] = 2
word_to_idx['<UNUSED>'] = 3
idx_to_word = {v: k for k, v in word_to_idx.items()}
list(idx_to_word.items())[:10]
print(' '.join(idx_to_word[idx] for idx in train_data[0]))

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wha

In [23]:
train_labels[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [24]:
def multi_hot_sequences(sequences, dimension):  #kodowanie gorąco jedynkowe
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0
    return results

train_data = multi_hot_sequences(train_data, dimension=NUM_WORDS)
test_data = multi_hot_sequences(test_data, dimension=NUM_WORDS)
train_data.shape

(25000, 10000)

In [25]:
test_data.shape

(25000, 10000)

### Budowa modelu bazowego

In [26]:
baseline_model = Sequential()
baseline_model.add(Dense(units=16, activation='relu', input_shape=(NUM_WORDS,) ))
baseline_model.add(Dense(units=16, activation='relu'))
baseline_model.add(Dense(units=1, activation='sigmoid'))

baseline_model.compile(optimizer='adam',
                       loss= 'binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])
baseline_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 16)                160016    
_________________________________________________________________
dense_14 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [35]:
baseline_history = baseline_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 [==============================] - 3s 61ms/step - loss: 0.6002 - accuracy: 0.6682 - binary_crossentropy: 0.6002 - val_loss: 0.5275 - val_accuracy: 0.8149 - val_binary_crossentropy: 0.5275
Epoch 2/20
49/49 [==============================] - 2s 35ms/step - loss: 0.3968 - accuracy: 0.8800 - binary_crossentropy: 0.3968 - val_loss: 0.3403 - val_accuracy: 0.8831 - val_binary_crossentropy: 0.3403
Epoch 3/20
49/49 [==============================] - 2s 34ms/step - loss: 0.2523 - accuracy: 0.9231 - binary_crossentropy: 0.2523 - val_loss: 0.2958 - val_accuracy: 0.8873 - val_binary_crossentropy: 0.2958
Epoch 4/20
49/49 [==============================] - 2s 35ms/step - loss: 0.1981 - accuracy: 0.9379 - binary_crossentropy: 0.1981 - val_loss: 0.2881 - val_accuracy: 0.8867 - val_binary_crossentropy: 0.2881
Epoch 5/20
49/49 [==============================] - 2s 35ms/step - loss: 0.1675 - accuracy: 0.9478 - binary_crossentropy: 0.1675 - val_loss: 0.2950 - val_accuracy: 0.8820 - val_bin

### Budowa mniejszego modelu


In [31]:
smaller_model = Sequential()
smaller_model.add(Dense(units=4, activation='relu', input_shape=(NUM_WORDS,) ))
smaller_model.add(Dense(units=4, activation='relu'))
smaller_model.add(Dense(units=1, activation='sigmoid'))

smaller_model.compile(optimizer='adam',
                       loss= 'binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])
smaller_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 4)                 40004     
_________________________________________________________________
dense_23 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 5         
Total params: 40,029
Trainable params: 40,029
Non-trainable params: 0
_________________________________________________________________


In [32]:
smaller_history = smaller_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 [==============================] - 3s 51ms/step - loss: 0.6134 - accuracy: 0.6496 - binary_crossentropy: 0.6134 - val_loss: 0.5521 - val_accuracy: 0.7642 - val_binary_crossentropy: 0.5521
Epoch 2/20
49/49 [==============================] - 2s 36ms/step - loss: 0.5002 - accuracy: 0.8261 - binary_crossentropy: 0.5002 - val_loss: 0.4985 - val_accuracy: 0.8242 - val_binary_crossentropy: 0.4985
Epoch 3/20
49/49 [==============================] - 2s 36ms/step - loss: 0.4480 - accuracy: 0.8852 - binary_crossentropy: 0.4480 - val_loss: 0.4733 - val_accuracy: 0.8485 - val_binary_crossentropy: 0.4733
Epoch 4/20
49/49 [==============================] - 2s 36ms/step - loss: 0.4127 - accuracy: 0.9115 - binary_crossentropy: 0.4127 - val_loss: 0.4568 - val_accuracy: 0.8635 - val_binary_crossentropy: 0.4568
Epoch 5/20
49/49 [==============================] - 2s 36ms/step - loss: 0.3844 - accuracy: 0.9293 - binary_crossentropy: 0.3844 - val_loss: 0.4485 - val_accuracy: 0.8669 - val_bin

### Budowa większego modelu

In [33]:
bigger_model = Sequential()
bigger_model.add(Dense(512, activation='relu', input_shape=(NUM_WORDS,)))
bigger_model.add(Dense(512, activation='relu'))
bigger_model.add(Dense(1, activation='sigmoid'))

bigger_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

bigger_model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 512)               5120512   
_________________________________________________________________
dense_26 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 513       
Total params: 5,383,681
Trainable params: 5,383,681
Non-trainable params: 0
_________________________________________________________________


In [34]:
bigger_history = bigger_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 [==============================] - 15s 307ms/step - loss: 0.3451 - accuracy: 0.8527 - binary_crossentropy: 0.3451 - val_loss: 0.2857 - val_accuracy: 0.8848 - val_binary_crossentropy: 0.2857
Epoch 2/20
49/49 [==============================] - 15s 302ms/step - loss: 0.1371 - accuracy: 0.9512 - binary_crossentropy: 0.1371 - val_loss: 0.3199 - val_accuracy: 0.8756 - val_binary_crossentropy: 0.3199
Epoch 3/20
49/49 [==============================] - 15s 301ms/step - loss: 0.0428 - accuracy: 0.9881 - binary_crossentropy: 0.0428 - val_loss: 0.4509 - val_accuracy: 0.8691 - val_binary_crossentropy: 0.4509
Epoch 4/20
49/49 [==============================] - 15s 300ms/step - loss: 0.0071 - accuracy: 0.9988 - binary_crossentropy: 0.0071 - val_loss: 0.5801 - val_accuracy: 0.8673 - val_binary_crossentropy: 0.5801
Epoch 5/20
49/49 [==============================] - 14s 297ms/step - loss: 0.0015 - accuracy: 1.0000 - binary_crossentropy: 0.0015 - val_loss: 0.6643 - val_accuracy: 0.8718

In [37]:
hist = pd.DataFrame(baseline_history.history)
hist['epoch'] = baseline_history.epoch
hist.head(20)

,loss,accuracy,binary_crossentropy,val_loss,val_accuracy,val_binary_crossentropy,epoch
0,0.600177,0.66820,0.600177,0.527457,0.81488,0.527457,0
1,0.396812,0.88004,0.396812,0.340291,0.88308,0.340291,1
2,0.252285,0.92312,0.252285,0.295832,0.88728,0.295832,2
3,0.198104,0.93788,0.198104,0.288055,0.88672,0.288055,3
4,0.167496,0.94780,0.167496,0.295049,0.88196,0.295049,4
5,0.146111,0.95568,0.146111,0.306006,0.87936,0.306006,5
6,0.129961,0.96128,0.129961,0.315095,0.87736,0.315095,6
7,0.115032,0.96656,0.115032,0.328606,0.87564,0.328606,7
8,0.103532,0.97096,0.103532,0.346622,0.87220,0.346622,8
9,0.093640,0.97536,0.093640,0.362853,0.87164,0.362853,9


### Porównanie wydajności modeli

In [40]:
import plotly.graph_objects as go

fig = go.Figure()
for name, history in zip(['smaller', 'baseline', 'bigger'], [smaller_history, baseline_history, bigger_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoch', yaxis_title='binary_crossentropy')
fig.show()

### Metody regularyzacji

In [42]:
from tensorflow.keras.regularizers import l2

l2_model = Sequential()
l2_model.add(Dense(16, kernel_regularizer=l2(0.001), activation='relu', input_shape=(NUM_WORDS,)))
l2_model.add(Dense(16, kernel_regularizer=l2(0.01), activation='relu'))
l2_model.add(Dense(1, activation='sigmoid'))

l2_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

l2_model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 16)                160016    
_________________________________________________________________
dense_29 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [43]:
l2_model_history = l2_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 [==============================] - 3s 50ms/step - loss: 0.6639 - accuracy: 0.8042 - binary_crossentropy: 0.4976 - val_loss: 0.5098 - val_accuracy: 0.8708 - val_binary_crossentropy: 0.3532
Epoch 2/20
49/49 [==============================] - 2s 39ms/step - loss: 0.4225 - accuracy: 0.9054 - binary_crossentropy: 0.2689 - val_loss: 0.4363 - val_accuracy: 0.8880 - val_binary_crossentropy: 0.2885
Epoch 3/20
49/49 [==============================] - 2s 39ms/step - loss: 0.3554 - accuracy: 0.9260 - binary_crossentropy: 0.2132 - val_loss: 0.4180 - val_accuracy: 0.8872 - val_binary_crossentropy: 0.2826
Epoch 4/20
49/49 [==============================] - 2s 39ms/step - loss: 0.3199 - accuracy: 0.9360 - binary_crossentropy: 0.1893 - val_loss: 0.4122 - val_accuracy: 0.8839 - val_binary_crossentropy: 0.2870
Epoch 5/20
49/49 [==============================] - 2s 39ms/step - loss: 0.2973 - accuracy: 0.9394 - binary_crossentropy: 0.1759 - val_loss: 0.4076 - val_accuracy: 0.8824 - val_bin

In [44]:
fig = go.Figure()
for name, history in zip(['baseline', 'l2'], [baseline_history, l2_model_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()

In [45]:
from tensorflow.keras.layers import Dropout

dropout_model = Sequential()
dropout_model.add(Dense(16, activation='relu', input_shape=(NUM_WORDS,)))
dropout_model.add(Dropout(0.5))
dropout_model.add(Dense(16, activation='relu'))
dropout_model.add(Dropout(0.5))
dropout_model.add(Dense(1, activation='sigmoid'))

dropout_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

dropout_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 16)                160016    
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_32 (Dense)             (None, 16)                272       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [46]:
dropout_history = dropout_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 [==============================] - 3s 47ms/step - loss: 0.6061 - accuracy: 0.6705 - binary_crossentropy: 0.6061 - val_loss: 0.4636 - val_accuracy: 0.8558 - val_binary_crossentropy: 0.4636
Epoch 2/20
49/49 [==============================] - 2s 38ms/step - loss: 0.4355 - accuracy: 0.8180 - binary_crossentropy: 0.4355 - val_loss: 0.3308 - val_accuracy: 0.8808 - val_binary_crossentropy: 0.3308
Epoch 3/20
49/49 [==============================] - 2s 38ms/step - loss: 0.3381 - accuracy: 0.8712 - binary_crossentropy: 0.3381 - val_loss: 0.2823 - val_accuracy: 0.8891 - val_binary_crossentropy: 0.2823
Epoch 4/20
49/49 [==============================] - 2s 38ms/step - loss: 0.2778 - accuracy: 0.9002 - binary_crossentropy: 0.2778 - val_loss: 0.2773 - val_accuracy: 0.8848 - val_binary_crossentropy: 0.2773
Epoch 5/20
49/49 [==============================] - 2s 38ms/step - loss: 0.2392 - accuracy: 0.9157 - binary_crossentropy: 0.2392 - val_loss: 0.2829 - val_accuracy: 0.8868 - val_bin

In [47]:
fig = go.Figure()
for name, history in zip(['baseline', 'dropout'], [baseline_history, dropout_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()

In [48]:
l2_model_dropout = Sequential()
l2_model_dropout.add(Dense(16, kernel_regularizer=l2(0.001), activation='relu', input_shape=(NUM_WORDS,)))
l2_model_dropout.add(Dropout(0.5))
l2_model_dropout.add(Dense(16, kernel_regularizer=l2(0.01), activation='relu'))
l2_model_dropout.add(Dropout(0.5))
l2_model_dropout.add(Dense(1, activation='sigmoid'))

l2_model_dropout.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

l2_model_dropout.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 16)                160016    
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_35 (Dense)             (None, 16)                272       
_________________________________________________________________
dropout_3 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [51]:
l2_dropout_history = l2_model_dropout.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 [==============================] - 4s 72ms/step - loss: 0.2492 - accuracy: 0.9505 - binary_crossentropy: 0.1501 - val_loss: 0.4138 - val_accuracy: 0.8788 - val_binary_crossentropy: 0.3148
Epoch 2/20
49/49 [==============================] - 2s 40ms/step - loss: 0.2525 - accuracy: 0.9474 - binary_crossentropy: 0.1534 - val_loss: 0.4195 - val_accuracy: 0.8794 - val_binary_crossentropy: 0.3202
Epoch 3/20
49/49 [==============================] - 2s 39ms/step - loss: 0.2520 - accuracy: 0.9478 - binary_crossentropy: 0.1524 - val_loss: 0.4211 - val_accuracy: 0.8784 - val_binary_crossentropy: 0.3217
Epoch 4/20
49/49 [==============================] - 2s 40ms/step - loss: 0.2472 - accuracy: 0.9516 - binary_crossentropy: 0.1477 - val_loss: 0.4340 - val_accuracy: 0.8768 - val_binary_crossentropy: 0.3347
Epoch 5/20
49/49 [==============================] - 2s 39ms/step - loss: 0.2482 - accuracy: 0.9492 - binary_crossentropy: 0.1488 - val_loss: 0.4283 - val_accuracy: 0.8768 - val_bin

In [53]:
fig = go.Figure()
for name, history in zip(['baseline', 'l2_dropout'], [baseline_history, l2_dropout_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()